# iMODSEAWAT: Henry case

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import imod
import subprocess
from collections import OrderedDict

%matplotlib inline

In [ ]:
nrow = 1
ncol = 100
nlay = 50

dz = 1.0
dx = 1.0
dy = dx

# scale parameters with discretization
qscaled = 0.03 * (dx * dz * dy)
dsp_al = 0.1 * dx
dsp_dmcoef = 0.05 * dx

We start by defining the domain, and setting up tops and bottoms.

In [ ]:
bnd = xr.DataArray(
    data = np.full((nrow, ncol, nlay), 1.0),
    coords = {"y": [0.5], "x": np.arange(0.5 * dx, dx * ncol, dx), "layer": np.arange(1, 1 + nlay)},
    dims = ("y", "x", "layer"),
)

This is a nice example of automatic broadcasting in xarray: because both `bnd` and `top1D` have the same coordinate `"layer"`, it knows how to broadcast to the other dimensions.

In [ ]:
top1D = xr.DataArray(np.arange(nlay * dz, 0.0, -dz), {"layer":np.arange(1, 1 + nlay)}, ("layer"))
top = bnd * top1D

# We define constant head here, after generating the tops, or we'd end up with negative top values
bnd[:,-1,:] = -1
# layer 0 is at the top, so we flip around the y-axis with yincrase
bnd.plot(y="layer", yincrease=False)

We need to force it into a transient run, so we assign a start and end time to the BND array.

In [ ]:
bnd_s = bnd.copy().assign_coords(time=np.datetime64("2000-01-01"))
bnd_e = bnd.copy().assign_coords(time=np.datetime64("2001-12-31"))
bnd_tr = xr.concat([bnd_s, bnd_e], dim="time")

Define WEL data: take care that columns are ordered properly! 

iMODSEAWAT does not use the column names, it relies on ordering only, which is:
1. x 
2. y
3. rate

Everything afterwards can be in arbitrary order.

In [ ]:
weldata = pd.DataFrame()
weldata["x"] = np.full(nlay, 0.5 * dx)
weldata["y"] = np.full(nlay, 0.5)
weldata["q"] = qscaled
weldata["layer"] = np.arange(1, 1 + nlay)
welconc = xr.full_like(bnd, np.nan)
welconc[:,0,:] = 0.0

Finally, we build up the model. The `imod` package uses the keys of the dictionary ("bnd", "shd", etc.) to determine what the data means.

In [ ]:
model = OrderedDict()
model["bnd"] = bnd_tr
model["icbund"] = xr.full_like(bnd, 1.0)
model["khv"] = xr.full_like(bnd, 10.0)
model["kva"] = xr.full_like(bnd, 10.0)
model["top"] = top
model["bot"] = top - dz
model["thickness"] = model["top"] - model["bot"]
model["shd"] = xr.full_like(bnd, 1.0)
model["sto"] = xr.full_like(bnd, 0.0001)
model["por"] = xr.full_like(bnd, 0.35)
model["sconc"] = xr.full_like(bnd, 35.0)
model["dsp-al"] = xr.full_like(bnd, dsp_al)
model["dsp-trpt"] = xr.full_like(bnd, 1.0)
model["dsp-trpv"] = xr.full_like(bnd, 1.0)
model["dsp-dmcoef"] = xr.full_like(bnd, dsp_dmcoef)
model["wel-rate"] = weldata
model["wel-conc"] = welconc

Now we write the model, including runfile:

In [ ]:
imod.seawat_write("HenryCase", model)

You can run the model from a jupyter notebook, by using the `subprocess` package. You won't get any output printed here however, so you'll only have the exit code to see if the run went right. 

Just replace "imod-wq_svn269_x64r.exe" with the path to whatever executable you're using:

In [ ]:
subprocess.run("imod-wq_svn270_x64r.exe HenryCase/runfile.run")

# Results

We can use the glob paths, with wildcards like `*`, to easily open the results IDFs for all layers:

In [ ]:
head = imod.idf.load(r"results\head_20011231000000_l*.idf")
head.plot(y="layer", yincrease=False)

In [ ]:
conc = imod.idf.load(r"results\conc_20011231000000_l*.idf")
conc.plot(y="layer", levels=range(0,35,5), yincrease=False)